<a href="https://colab.research.google.com/github/DaviRamos/NLP/blob/master/Spacy3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tokenization
Uma boa forma de começar a analisar o texto é o separando em unidades menores. O nome desse processo é tokenization, que basicamente significa separar o texto em palavras.

No spaCy você utiliza models, que são tipo os ‘cérebros treinados’ de cada língua. A biblioteca ainda não tem um model para Português :(, só para Inglês e Alemão.

In [1]:
pip install -U Spacy

Requirement already up-to-date: Spacy in /usr/local/lib/python3.6/dist-packages (2.1.4)


In [2]:
import spacy.cli
spacy.cli.download("en")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [0]:
nlp = spacy.load('en')

In [0]:
raw_text = "Seven years after the death of his wife, Mill was invited to contest Westminster. His feeling on the conduct of elections made him refuse to take any personal action in the matter, and he gave the frankest expression to his political views, but nevertheless he was elected by a large majority. He was not a conventional success in the House; as a speaker he lacked magnetism. But his influence was widely felt."

In [0]:
parsedData = nlp(raw_text) 

Ao passar o texto para o model ele já o separa em tokens e computa várias outras propriedades.

In [10]:
word = parsedData[0]
print(word.text, word.lower_)

Seven seven


# Part-of-speech tagging
Lembra na escola quando a gente tinha que identificar quem era o substantivo, o verbo e os adjetivos nas frases? Em NLP esse processo se chama part-of-speech tagging. É a boa e velha análise gramatical.

O model já fez essa análise, que pode ser acessada pela propriedade.pos_ de cada token.

In [11]:
for i,word in enumerate(parsedData):
    print(word.text, word.pos_)
    if i > 5:
        break

Seven NUM
years NOUN
after ADP
the DET
death NOUN
of ADP
his DET


[Aqui](https://spacy.io/usage/linguistic-features#pos-tagging) a gente pode ver a lista com o significado de cada uma dessas tags.

Existe também uma tag mais elaborada, que pode ser acessada pela propriedade .tag_ Essa tag contém informações sobre a estrutura morfológica da palavra.

In [12]:
word = parsedData[10] #a palavra 'was'
print("original:",word.text)
print("POS tag:",word.pos_)
print("fine grainned POS tag:", word.tag_)

original: was
POS tag: VERB
fine grainned POS tag: VBD


# Named Entity Recognition (NER)
Esse processo busca identificar categorias como nomes de pessoas, organizações, locais, porcentagens, valores monetários, e por ai vai. Essas categorias podem ser pré-definidas por nós, então dependendo do texto podemos criar nossas próprias entidades.

Pra variar, o model também já computou isso pra a gente na propriedade .ent_type_. Vamos ver quantas entidades nosso texto tem:

In [13]:
for word in parsedData:
    if word.ent_type_:
        print(word.text, word.ent_type_)

Seven DATE
years DATE
Mill PERSON
Westminster PERSON
House ORG


# Syntactic Parsing
Syntactic parsing é o processo de representar as dependências do texto através das relações entre os tokens. Por exemplo: um artigo está ligado a um substantivo, um advérbio modifica um verbo, e por ai vai.

O[ Explosion AI ](https://explosion.ai/)criou uma ferramenta linda para visualizar essas dependências, o [Dependency Visualizer](https://explosion.ai/demos/displacy).

O atributo para ter acesso à dependência sintática de cada token é o .dep_

In [14]:
for word in parsedData:
    print(word.text, word.dep_)

Seven nummod
years npadvmod
after prep
the det
death pobj
of prep
his poss
wife pobj
, punct
Mill nsubjpass
was auxpass
invited ROOT
to aux
contest xcomp
Westminster dobj
. punct
His poss
feeling nsubj
on prep
the det
conduct pobj
of prep
elections pobj
made ROOT
him nsubj
refuse ccomp
to aux
take xcomp
any det
personal amod
action dobj
in prep
the det
matter pobj
, punct
and cc
he nsubj
gave conj
the det
frankest amod
expression dobj
to dative
his poss
political amod
views pobj
, punct
but cc
nevertheless advmod
he nsubjpass
was auxpass
elected conj
by agent
a det
large amod
majority pobj
. punct
He nsubj
was ccomp
not neg
a det
conventional amod
success attr
in prep
the det
House pobj
; punct
as prep
a det
speaker pobj
he nsubj
lacked ROOT
magnetism dobj
. punct
But cc
his poss
influence nsubjpass
was auxpass
widely advmod
felt ROOT
. punct


Existem várias formas de navegar pela árvore sintática. [Aqui na documentação](https://spacy.io/usage/linguistic-features#dependency-parse) eles explicam tudo que pode ser feito. É possível encontrar um verbo e o sujeito dele, por exemplo.

[Aqui](https://emorynlp.github.io/nlp4j/components/dependency-parsing.html) tem uma lista com o significado de cada tag da análise sintática e nesse [site aqui](https://universaldependencies.org/en/dep/) eles explicam mais o significado de cada termo.

Agora vamos ver funcionalidades do spaCy que nos ajudam a trabalhar com as métricas que vimos até aqui.

# Rule-based matching
Esse rule-based matching permite a criação de regras utilizando os atributos e flags dos tokens. Isso pode ser bem interessante nos casos onde a gente tá trabalhando em um corpus sobre um domínio específico.

Por exemplo, eu quero encontrar todos os sujeitos nominais do texto:

In [0]:
from spacy.attrs import DEP
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)
matcher.add_pattern("SujeitoNominal", [ {DEP:'nsubj'}])

In [0]:
doc = nlp(raw_text)
matches = matcher(doc)

In [0]:
for ent_id, label, start, end in matcher(doc):
    print(doc[start:end].text)

# Ainda é possível:

[Criar uma regra nova para a extração dos tokens](https://spacy.io/usage/linguistic-features#tokenization)

[Criar uma classe Tokenizer do zero](https://spacy.io/usage/linguistic-features#tokenization)

[Usar um tokenizer arbitrário em um model](https://spacy.io/usage/linguistic-features#tokenization)

# Word vectors
Para computar as similaridades entre palavras uma técnica comum é representá-las através de vetores. A forma mais famosa de treinar esses vetores é com a família de algoritmos do [word2vec](https://en.wikipedia.org/wiki/Word2vec).

Com o spaCy é fácil trabalhar com vetores porque as classes Lexeme, Token, Span e Doc têm uma propriedade .vector . Vamos testar isso vendo as similaridades entre um cachorro, um gato e um cavalo.

In [18]:
my,dog,and_,cat,and__,horse = nlp(u'my dog and cat and horse')
print(cat.similarity(dog))
print(cat.similarity(horse))
print(dog.similarity(horse))

0.61500615
0.67795044
0.5448442


/usr/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instea